# Prerequisite

## Libraries Preparation

In [1]:
import os
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras import backend as K
from keras.optimizers import SGD, Adagrad, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Reshape, Activation, Multiply,GlobalMaxPooling2D
from keras.layers import LSTM, TimeDistributed, Dropout, Input, Flatten, Lambda, Concatenate, Average, Permute
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import concatenate
from keras.regularizers import l1, l2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Using TensorFlow backend.


## Aestethic Purposes

In [2]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
def ensure_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

## Data Generator 

In [3]:
def get_generators(batch_size):
    train_data_gen = ImageDataGenerator(rotation_range=20.,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    rescale=1. / 255)
    valid_data_gen = ImageDataGenerator(rescale=1. / 255)
    train_generator = train_data_gen.flow_from_directory(
          'data/train',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode="categorical",
          shuffle=True)

    validation_generator = valid_data_gen.flow_from_directory(
          'data/valid',
          target_size=(224, 224),
          batch_size=batch_size,  
          class_mode="categorical",
          shuffle=True)
    return train_generator, validation_generator


## Create Densenet121 Model

In [4]:
def get_model_densenet121(num_classes,weights='imagenet'):
    # create the base pre-trained model
    base_model = DenseNet121(weights=weights, include_top=False)
    x = base_model.output
    # add a global spatial average pooling layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## 1st Fine-Tune (Unfreeze Top 2 Block)

In [5]:
def fine_tune_dense_layer(model):
    """After we fine-tune the dense layers, train deeper."""
    # we chose to train the top 2 dense blocks, i.e. we will freeze
    # the first 140 layers and unfreeze the rest:

    for layer in model.layers[:143]:
        layer.trainable = False
    for layer in model.layers[143:]:
        layer.trainable = True
    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0005, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    return model


## 2nd Fine-Tune (Unfreeze All Block)

In [6]:
def fine_tune_all_layer(model):
   # unfreeze all dense layers to train. 
    for layer in model.layers:
        layer.trainable = True
    # we need to recompile the model for these modifications to take effect
    # we use Adagrad for faster convergence.
    model.compile(
        optimizer=Adagrad(lr=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    return model


## Training parameters

In [7]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=816, #This should be changed depending on your batch size. Generally it should be sample_number/batch_size
        validation_data=validation_generator,
        validation_steps=100, #This should be changed depending on your batch size. Generally it should be sample_number/batch_size
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

# Training and Validation Phase

## First Stage of Training

In the first stage, we freeze all layer for training except for the top most layer. This will allow the model to learn using the pretrained weight from imagenet

In [ ]:
#Remove existing model to avoid mixing
import os, shutil
folder = 'checkpoints' #Folder name to save model checkpoints 
ensure_folder(folder)

for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

# Helper: Stop when we stop learning.
# patience: number of epochs with no improvement after which training will be stopped.
early_stopper = EarlyStopping(patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.05, patience=5, verbose=1)
# Helper: TensorBoard
tensorboard = TensorBoard(log_dir='./data/logs/')

weights_file = None
num_classes = 196
batch_size = 16
nb_epoch = 20
model = get_model_densenet121(num_classes) #Densenet121
print(model.summary())
generators = get_generators(batch_size)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
print(pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).iloc[:,1:3].to_string())
# Helper: Save the min val_loss model in each epoch.
checkpointer = ModelCheckpoint(
    filepath='checkpoints/model.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}.hdf5',
    monitor='val_acc',
    verbose=1,
    save_best_only=True)

if weights_file is None:
    print(color.BOLD +"Training Top layers."+ color.END)
    model = train_model(model,nb_epoch, generators,[checkpointer])
else:
    print("Loading saved model: %s." % weights_file)
    model.load_weights(weights_file)
    

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
_____________________________________

Found 13030 images belonging to 196 classes.
Found 1629 images belonging to 196 classes.
                     Layer Name  Layer Trainable
0                       input_1            False
1              zero_padding2d_1            False
2                    conv1/conv            False
3                      conv1/bn            False
4                    conv1/relu            False
5              zero_padding2d_2            False
6                         pool1            False
7             conv2_block1_0_bn            False
8           conv2_block1_0_relu            False
9           conv2_block1_1_conv            False
10            conv2_block1_1_bn            False
11          conv2_block1_1_relu            False
12          conv2_block1_2_conv            False
13          conv2_block1_concat            False
14            conv2_block2_0_bn            False
15          conv2_block2_0_relu            False
16          conv2_block2_1_conv            False
17            conv2_block2_1_

429                     dense_2             True
Training Top layers.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
816/816 [==============================] - 184s 225ms/step - loss: 4.6483 - acc: 0.0620 - val_loss: 4.6306 - val_acc: 0.0500

Epoch 00001: val_acc improved from -inf to 0.05000, saving model to checkpoints/model.001-4.63-0.05.hdf5
Epoch 2/20
816/816 [==============================] - 179s 219ms/step - loss: 3.4421 - acc: 0.2013 - val_loss: 4.6792 - val_acc: 0.0776

Epoch 00002: val_acc improved from 0.05000 to 0.07765, saving model to checkpoints/model.002-4.68-0.08.hdf5
Epoch 3/20
816/816 [==============================] - 179s 220ms/step - loss: 2.9286 - acc: 0.2952 - val_loss: 4.8068 - val_acc: 0.0939

Epoch 00003: val_acc improved from 0.07765 to 0.09393, saving model to checkpoints/model.003-4.81-0.09.hdf5
Epoch 4/20
816/816 [==============================] - 179s 219ms/step - loss: 2.5964 - acc: 0.3570 - val_loss: 4.4615 - val_acc: 0.1446

Epoch 00004: 

## Second stage of training

In the second stage of training, we fine-tune the model by unfreeze the last 2 block of the densenet and freeze the rest.

In [ ]:
#Load the highest accuracy model
moddir = os.listdir(folder)
model.load_weights(os.path.join('checkpoints',moddir[len(moddir)-1]))

# Get and train the mid layers.
checkpointer_finetuned = ModelCheckpoint(
    filepath='checkpoints/model_finetuned.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}.hdf5',
    monitor='val_acc',
    verbose=1,
    save_best_only=True)
model = fine_tune_dense_layer(model) #Dense121
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
print(pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).iloc[:,1:3].to_string())

nb_epoch = 80
print()
print(color.BOLD + "Fine-tuning Mid Layers" + color.END)
model = train_model(model, nb_epoch, generators,
                    [checkpointer_finetuned, early_stopper, tensorboard, reduce_lr])

                     Layer Name  Layer Trainable
0                       input_1            False
1              zero_padding2d_1            False
2                    conv1/conv            False
3                      conv1/bn            False
4                    conv1/relu            False
5              zero_padding2d_2            False
6                         pool1            False
7             conv2_block1_0_bn            False
8           conv2_block1_0_relu            False
9           conv2_block1_1_conv            False
10            conv2_block1_1_bn            False
11          conv2_block1_1_relu            False
12          conv2_block1_2_conv            False
13          conv2_block1_concat            False
14            conv2_block2_0_bn            False
15          conv2_block2_0_relu            False
16          conv2_block2_1_conv            False
17            conv2_block2_1_bn            False
18          conv2_block2_1_relu            False
19          conv2_bl

Epoch 1/80
816/816 [==============================] - 197s 242ms/step - loss: 1.2111 - acc: 0.6703 - top_k_categorical_accuracy: 0.9078 - val_loss: 3.2643 - val_acc: 0.4152 - val_top_k_categorical_accuracy: 0.7295

Epoch 00001: val_acc improved from -inf to 0.41515, saving model to checkpoints/model_finetuned.001-3.26-0.42.hdf5
Epoch 2/80
816/816 [==============================] - 189s 231ms/step - loss: 0.5612 - acc: 0.8275 - top_k_categorical_accuracy: 0.9771 - val_loss: 2.7008 - val_acc: 0.4872 - val_top_k_categorical_accuracy: 0.7865

Epoch 00002: val_acc improved from 0.41515 to 0.48716, saving model to checkpoints/model_finetuned.002-2.70-0.49.hdf5
Epoch 3/80
816/816 [==============================] - 191s 234ms/step - loss: 0.3537 - acc: 0.8893 - top_k_categorical_accuracy: 0.9898 - val_loss: 2.8073 - val_acc: 0.4703 - val_top_k_categorical_accuracy: 0.7708

Epoch 00003: val_acc did not improve from 0.48716
Epoch 4/80
816/816 [==============================] - 193s 236ms/step - 

## Third stage of training

In the third stage of training, we further fine-tune the model by unfreeze every layer on the dense layer. This is to allow the features learned in the last 2 block of densenet to affect the rest of the layers.

In [ ]:
nb_epoch = 120
moddir = os.listdir(folder)
model.load_weights(os.path.join('checkpoints',moddir[len(moddir)-1]))

# Get and train the mid layers.
checkpointer_all_finetuned = ModelCheckpoint(
    filepath='checkpoints/model_all_finetuned.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}.hdf5',
    monitor='val_acc',
    verbose=1,
    save_best_only=True)
early_stopper = EarlyStopping(patience=20)
model = fine_tune_all_layer(model)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
print(pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).iloc[:,1:3].to_string())
print()
print(color.BOLD + "Fine-tuning All Layers" + color.END)
model = train_model(model, nb_epoch, generators,
                    [checkpointer_all_finetuned, early_stopper, tensorboard, reduce_lr])

### Move the Best Model into saved_models Folder to Avoid Overwritting

The code is designed to overwrite all model checkpoint in each training session. You need to move the desired saved model into saved_models folder to avoid overwritting